# Basic Iterative Method

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torchvision import models
from torchvision import transforms

plt.rcParams['figure.figsize'] = (20.0, 10.0)
from IPython.display import clear_output

# Import helper functions
from modules.helper import *
from modules.dataset import ImageNetSubset
from modules.bim import apply_BIM
from modules.bim import compute_all_bim
from modules.bim import BIM_attack_with_selected_samples

## 1. Load Model and Data

We load the pre-trained googleNet and the data. For details see notebook `00_Helper-Functions`.

In [2]:
## Load model and data
model = models.googlenet(pretrained=True)
model.eval();

In [3]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]   

preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

data_loader = torch.utils.data.DataLoader(
    ImageNetSubset("data/ImageNet_subset//dev_dataset.csv", "data/ImageNet_subset//images/", transform=preprocess))

## 2. Attack

BIM requires the following function, which clips pixel values of an image $X^\prime$:

For the implementation we do for the number of iterations `n`:


a) Compute $ X^\prime = \widetilde{X}_{n} + \alpha sign(\nabla_{X} J(\widetilde{X}_{n}, Y_{true}))$ as used in equation (2.1)

b) Compute $ X(x, y, z) + \epsilon $ and $ X(x, y, z) - \epsilon $

c) Evaluate equation (2.2) using steps a) and b)

$max\{0, X(x, y, z)-\epsilon, X^\prime(x, y, z) \}$ (?)

d) Retrieve updated adversarial image $\widetilde{X}_{1}$ as given in equation (2.1)

Values for the hyper parameters given in [2]:

- $\alpha = \frac{1}{255}$

- Number of iterations: $min(4+\epsilon, 1.25 \cdot \epsilon)$

- $\epsilon $

In [ ]:
## TODO: CHECK alpha
def compare_examples_bim(alpha, epsilon, idx, folder=None):
    '''
    Generates an example using BIM. Prints infos and plots clean and adversarial image side-by-side.
    
    Inputs:
    alpha   -- Hyperparameter for BIM
    epsilon -- Hyperparameter for BIM
    idx     -- Index of sample   
    folder  -- If given image will be saved to this folder
    '''
    
    num_iterations = int(np.min([np.ceil( (epsilon/alpha) + 4 ), np.ceil( 1.25 * epsilon/alpha ) ]))
    print("Number of iterations: " + str(num_iterations))
    
    
    # Get data
    image_clean, target_class = data_loader.dataset[idx]
    image_clean.unsqueeze_(0)
    target_class.unsqueeze_(0)

    # Predict clean example
    labels, confidences, gradient = predict(model, image_clean, target_class, return_grad=True)
    label_clean = labels[0]
    conf_clean = confidences[0]
    
    # Compute adversarial image and predict for it.    
    image_adv = apply_BIM(model, mean, std, image_clean, target_class, alpha, epsilon, num_iterations=num_iterations)
    labels, confidences, _ = predict(model, image_adv, target_class, return_grad=False)
    label_adv = labels[0]
    conf_adv = confidences[0]
    
    
    # Plot
    summarize_attack(image_clean, image_adv, conf_clean, conf_adv, label_clean, label_adv, target_class, idx,
                        folder=folder)

### 2.1 Individual Images

Do some cool analysis here. In particularm find one of the examples where a wrong class is predicted with high confidence. Apparently there aren't neccessarily that many in the ImageNet dataset?

## 2. Attack

In [ ]:
alpha = 1/255
epsilon = 16/255

compare_examples_bim(alpha, epsilon, 766, folder=None)

## 0. Introduction

## 1. All Images

We are splitting the data according to the original confidences. For each of the intervals we craft adversarial examples at varying epsilon and report the confidence and accuracy of these attacks for the interval.


### 1.1 Intervals of initial confidences

use function `BIM_attack_with_selected_samples`

In [ ]:
# Phil -> Done
max_confidence = 0.60
min_confidence = 0.40


# Hyperparameter for attacks
epsilons = [0, 0.5/255, 1/255, 2/255, 4/255, 10/255, 20/255, 30/255, 40/255, 50/255, 60/255]    
alpha = 1/255

result = BIM_attack_with_selected_samples(min_confidence, max_confidence, data_loader, predict, model, mean, std, epsilons, alpha)

In [4]:
# Dave -> Done
max_confidence = 0.90
min_confidence = 0.85


# Hyperparameter for attacks
epsilons = [0, 0.5/255, 1/255, 2/255, 4/255, 10/255, 20/255, 30/255, 40/255, 50/255, 60/255] 
alpha = 1/255

result = BIM_attack_with_selected_samples(min_confidence, max_confidence, data_loader, predict, model, mean, std, epsilons, alpha)

Running for epsilon 60.00
Sample: 69 of 69
Accuracy top 1 adversarial: 0.00
Accuracy top 5 adversarial: 0.00
Confidence adversarial: 0.72
Number of iterations: 64


In [ ]:
# Phil -> Done
max_confidence = 0.95
min_confidence = 0.90


# Hyperparameter for attacks
epsilons = [0, 0.5/255, 1/255, 2/255, 4/255, 10/255, 20/255, 30/255, 40/255, 50/255, 60/255]    
alpha = 1/255

result = BIM_attack_with_selected_samples(min_confidence, max_confidence, data_loader, predict, model, mean, std, epsilons, alpha)

In [ ]:
# Phil -> In progress
max_confidence = 0.99
min_confidence = 0.95


# Hyperparameter for attacks
epsilons = [0, 0.5/255, 1/255, 2/255, 4/255, 10/255, 20/255, 30/255, 40/255, 50/255, 60/255]    
alpha = 1/255

result = BIM_attack_with_selected_samples(min_confidence, max_confidence, data_loader, predict, model, mean, std, epsilons, alpha)

In [ ]:
# Dave -> Done

max_confidence = 1.00
min_confidence = 0.99


# Hyperparameter for attacks
epsilons = [0, 0.5/255, 1/255, 2/255, 4/255, 10/255, 20/255, 30/255, 40/255, 50/255, 60/255]    
alpha = 1/255

result = BIM_attack_with_selected_samples(min_confidence, max_confidence, data_loader, predict, model, mean, std, epsilons, alpha)

In [ ]:
### PLOT 1
####### PLOT 3
# Load data
result_99 = pd.read_csv("results/BIM-Conf99.csv")
result_95 = pd.read_csv("results/BIM-Conf95.csv")
result_90 = pd.read_csv("results/BIM-Conf90.csv")
result_85 = pd.read_csv("results/BIM-Conf85.csv") 
result_40 = pd.read_csv("results/BIM-Conf40.csv")

# Plot
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(20,10))

## Top left: Accuracies
axs[0].plot(result_99["Epsilon_255"], result_99["Accuracy Top 1"], "-^", color='navy', label='99% - 100%')
axs[0].plot(result_95["Epsilon_255"], result_95["Accuracy Top 1"], "-o", color='navy', label='95% - 99%')
axs[0].plot(result_90["Epsilon_255"], result_90["Accuracy Top 1"], "-*", color='navy', label='90% - 95%')
axs[0].plot(result_85["Epsilon_255"], result_85["Accuracy Top 1"], "-s", color='navy', label='85% - 90%')
axs[0].plot(result_40["Epsilon_255"], result_40["Accuracy Top 1"], "-v", color='navy', label='40% - 60%')
axs[0].legend()
#axs[0].set_title("Accuracies", fontsize=25)
axs[0].set_xlabel("Epsilon * 255", fontsize=20)

## Bottom left: Confidences
axs[1].plot(result_99["Epsilon_255"], result_99["Confidence"], "-^", color='orange', label='99% - 100%')
axs[1].plot(result_95["Epsilon_255"], result_95["Confidence"], "-o", color='orange', label='95% - 99%')
axs[1].plot(result_90["Epsilon_255"], result_90["Confidence"], "-*", color='orange', label='90% - 95%')
axs[1].plot(result_85["Epsilon_255"], result_85["Confidence"], "-s", color='orange', label='85% - 90%')
axs[1].plot(result_40["Epsilon_255"], result_40["Confidence"], "-v", color='orange', label='40% - 60%')
axs[1].legend()
#axs[1].set_title("Confidences", fontsize=25)
axs[1].set_xlabel("Epsilon * 255", fontsize=20)

## Save plot
fig.tight_layout()
fig.savefig("plots/BIM/Accuracies_Confidences.pdf")
fig.savefig("plots/BIM/Accuracies_Confidences.png")

## 2. Indivdual Images

In [ ]:
## TODO: CHECK alpha
#def get_confidence_acc_class(sample, epsilon, alpha, num_iterations):
    '''
    Computes BIM attack and returns info about success.
    ...
    
    Inputs:
    sample         -- Index of sample 
    epsilon        -- Hyperparameter for BIM
    alpha          -- Hyperparamezer for BIM
    num_iterations -- Number of iterations to perform the BIM with
    
    Returns:
    conf_adv       -- Confidence of adversary
    corr           -- Integer to indicate if predicted class is correct (1) or not (0)
    class_name_adv -- Label of adversarial class
    '''
       
    
    # Get data
    image_clean, class_index = data_loader.dataset[sample]
    image_clean.unsqueeze_(0)
    class_index.unsqueeze_(0)

    # Predict clean example
    _, _, gradient = predict(model, image_clean, class_index, return_grad=True)
              
    # Compute adversarial image and predict for it.
    image_adv = apply_BIM(model, mean, std, image_clean, class_index, alpha, epsilon, num_iterations=num_iterations)    
    predicted_classes, confidences, _ = predict(model, image_adv, class_index, return_grad=False)
    
    
    if predicted_classes[0] == class_index.squeeze().numpy():
        corr_adv = 1
    else: 
        corr_adv = 0
        
    conf_adv = confidences[0] 
    class_name_adv = idx_to_name(predicted_classes[0])
        
    return conf_adv, corr_adv, class_name_adv

In [ ]:
## TODO: CHECK alpha
#def analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=None, save_plot=False, print_output=True):
    '''
    Generates 4 plots: Image, conf over epsilon, top 5 conf for clean image, top 5 conf for adversarial image.
    '''  

    # Get data
    image_clean, class_index = data_loader.dataset[sample]
    image_clean.unsqueeze_(0)
    class_index.unsqueeze_(0)

    # Predict clean example
    _, confidences_clean, gradient = predict(model, image_clean, class_index, return_grad=True)
    
    epsilons = [0, 0.5/255, 1/255, 2/255, 4/255, 8/255, 12/255, 16/255, 20/255]

    
    conf_list = []
    acc_list = []

    print("Epsilon \t Accuracy \t Confidence \t Label")

    for epsilon in epsilons:
        if num_iterations == None:
            num_iterations = int(np.min([np.ceil( (epsilon/alpha) + 4 ), np.ceil( 1.25 * epsilon/alpha ) ]))
        
        conf_adv, acc, predicted_label = get_confidence_acc_class(sample, epsilon, alpha, num_iterations)
        conf_list.append(conf_adv)
        acc_list.append(acc)
        
        if print_output == True:
            print(str(epsilon*255) + "\t\t\t" + str(acc) + "\t" + str(conf_adv) + "\t" + predicted_label) 
    
    # Compute top 5 confidences for selected epsilon
    ## Number of iterations
    if num_iterations == None:
        num_iterations = int(np.min([np.ceil(epsilon*255*4), np.ceil(1.25+(epsilon*255))]))
    
    image_adv = apply_BIM(model, mean, std, image_clean, class_index, alpha, epsilon_conf, num_iterations=num_iterations)    
    
    _, confidences_adv, _ = predict(model, image_adv, class_index, return_grad=False)
    
     
    # Plot
    samples = [1, 2, 3, 4, 5]
    
    fig, axs = plt.subplots(1, 4, figsize=(20,5))

    ## First image: Clean image
    im = show_tensor_image(image_clean)

    axs[0].imshow(im)
    axs[0].axis('off')

    ## Second image: Confidence dist over epsilon and correct/incorrect
    axs[1].plot(np.array(epsilons)*255, conf_list, "-^", color='orange', label='Confidence')
    axs[1].plot(np.array(epsilons)*255, acc_list, "s", color='navy', label='1: Corr, 0: False')
    axs[1].set_ylim(0, 1.1)
    axs[1].set_xlabel("Epsilon *255", fontsize=10)
    axs[1].legend()

    ## Third image: Clean image top 5 confidence
    axs[2].bar(samples, confidences_clean, color='orange')
    axs[2].set_ylim(0, 1.1)
    axs[2].set_xlabel("Epsilon *255", fontsize=10)

    ## Fourth image: Adversarial image selected epsilon top 5 confidence
    axs[3].bar(samples, confidences_adv, color='orange')
    axs[3].set_ylim(0, 1.1)
    axs[3].set_xlabel("Epsilon *255", fontsize=10)
    
    if save_plot is True:
        fig.tight_layout()
        fig.savefig("plots/BIM/Individual_Images-Sample_" + str(sample) + ".png")

### 2.1 Confidence and correct/false class over epsilon for individual images
Can you find one which:
- Cannot be forced to change the class
- Changes class and then changes back?
- Has higher adversarial confidence than initial confidence

In [ ]:
# All images: Only for correct initial classifiation we compute adversaries.
epsilons = [1/255, 2/255, 4/255, 6/255, 8/255, 10/255]


result = pd.DataFrame()

for epsilon in epsilons:   
    conf = []
    
    for idx in range(1000):
        clear_output(wait=True)
        print("Epsilon " + str(epsilon*255))
        print(idx+1)
        conf_adv, acc, label_adv = get_confidence_acc_class(epsilon, idx)
        
        if acc == 1:
            conf.append(conf_adv)

        else:
            conf.append(0)

    result["Epsilon " + str(int(epsilon*255))] = conf

#result.to_csv("results/BIM-accuracies-corr_only.csv")

In [ ]:
result = pd.read_csv("results/BIM-accuracies-corr_only.csv", index_col=0)
result

#### 2.1.1 Predicted class invariant

#### Increasing interations

In [ ]:
sample = 132
alpha = 1
num_iter = 1
epsilon_conf = 4/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)


In [ ]:
sample = 132
alpha = 1
num_iter = 5
epsilon_conf = 4/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)

In [ ]:
sample = 132
alpha = 1
num_iter = 10
epsilon_conf = 4/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)

In [ ]:
sample = 132
alpha = 1
num_iter = 20
epsilon_conf = 4/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)

#### Lowering alpha

In [ ]:
sample = 132
alpha = 127/255
num_iter = 2
epsilon_conf = 4/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)

In [ ]:
sample = 132
alpha = 68/255
num_iter = 2
epsilon_conf = 4/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)

In [ ]:
sample = 132
alpha = 34/255
num_iter = 2
epsilon_conf = 4/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)

In [ ]:
sample = 132
alpha = 10/255
num_iter = 2
epsilon_conf = 4/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)

#### Combining alpha and iterations

In [ ]:
sample = 132
alpha = 127/255
num_iter = 15
epsilon_conf = 4/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)

In [ ]:
sample = 132
alpha = 68/255
num_iter = 15
epsilon_conf = 4/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)

In [ ]:
sample = 132
alpha = 34/255
num_iter = 15
epsilon_conf = 4/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)

In [ ]:
sample = 132
alpha = 10/255
num_iter = 15
epsilon_conf = 4/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)

In [ ]:
## Make subplot 3x3




## Apron example

In [ ]:
sample = 432
alpha = 10/255
num_iter = 15
epsilon_conf = 0/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)

In [ ]:
sample = 323
alpha = 10/255
num_iter = 15
epsilon_conf = 2/255
analyze_bim_conf(sample, alpha, epsilon_conf, num_iterations=num_iter, save_plot=False, print_output=False)

#### 2.1.2 Class bounces back

In [ ]:
sample = 766
epsilon_conf = 4/255
analyze_bim_conf(sample, epsilon_conf, save_plot=True)

In [ ]:
sample = 766
epsilon_conf = 4/255
analyze_bim_conf(sample, epsilon_conf, save_plot=True)

In [ ]:
sample = 766
epsilon_conf = 5/255
analyze_bim_conf(sample, epsilon_conf, save_plot=False)

In [ ]:
sample = 766
epsilon_conf = 18/255
analyze_bim_conf(sample, epsilon_conf, save_plot=False)

In [ ]:
sample = 432
epsilon_conf = 0.5/255
analyze_bim_conf(sample, epsilon_conf, save_plot=True)

#### 2.1.3 Adversarial confidence greater than initial confidence

In [ ]:
#--------> Find more examples!!!!!

In [ ]:
sample = 258
epsilon_conf = 8/255
analyze_bim_conf(sample, epsilon_conf, save_plot=True)

# Confidence up to 94% for wrong class at epsilon=8

## Some old stuff!!!!!

In [ ]:
iterate_epsilons(766)

In [ ]:
iterate_epsilons(766)

In [ ]:
iterate_epsilons(590)

In [ ]:
iterate_epsilons(132)

In [ ]:
iterate_epsilons(241)

### 2.2 All Images

We now run attacks with the whole dataset for different values of $\epsilon$. We will first, make predictions on clean images. Only if this prediction is correct we craft an adversarial example and attack the network. The networks accuracy on the clean dataset is $???$ as shown in the notebook `00_Helper-Functions`.

#### Comparison paper p.5
FGSM vs. BIM when including false initial predictions.

In [ ]:
## Part 1 -> Colab 1: Running
epsilons = [0, 1/255, 2/255, 4/255, 8/255, 16/255, 32/255]

alpha = 1/255 # recommended in paper
filename_ext = "0-32"

top1, top5, conf = compute_all_bim(model, data_loader, predict, mean, std, epsilons, alpha, filename_ext)

In [ ]:
## Part 2 -> Colab 2: Running
epsilons = [48/255, 64/255]

alpha = 1/255 # recommended in paper
filename_ext = "48-64"

top1, top5, conf = compute_all_bim(model, data_loader, predict, mean, std, epsilons, alpha, filename_ext)

In [ ]:
## Part 3 -> HP 1: Running
epsilons = [96/255]

alpha = 1/255 # recommended in paper
filename_ext = "96"

top1, top5, conf = compute_all_bim(model, data_loader, predict, mean, std, epsilons, alpha, filename_ext)

In [ ]:
## Part 4 -> Colab 3: Running
epsilons = [128/255]

alpha = 1/255 # recommended in paper
filename_ext = "128"

top1, top5, conf = compute_all_bim(model, data_loader, predict, mean, std, epsilons, alpha, filename_ext)

In [ ]:
plt.plot(conf, color='navy')

In [ ]:
epsilons = [0, 1/255, 2/255, 4/255, 8/255, 16/255, 32/255, 48/255, 64/255, 96/255, 128/255]
top1, top5, conf = compute_all_bim(model, data_loader, predict, mean, std, epsilons)

In [ ]:
plt.plot(np.array(epsilons)*255, top1, "-o", color='navy', label='Top 1')
plt.plot(np.array(epsilons)*255, top5, "-o", color='navy', label='Top 5')
plt.ylim(0, 1)
plt.legend()
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
plt.plot(np.array(epsilons)*255, conf, "-*", color='orange', label='Confidence')
plt.ylim(0, 1)
plt.legend()
plt.xlabel("Epsilon")
plt.ylabel("Confidence")
plt.show()

## Comparison: FGSM BIM and ILLM

In [ ]:
fgsm = pd.read_csv("results/FGSM/FGSM-all_samples.csv", index_col=0)
bim = pd.read_csv("results/BIM/BIM-all_samples.csv", index_col=0)
bim

In [ ]:
# Plot
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(20,5))

## Left: Accuracies
axs[0].plot(fgsm["Epsilon"], fgsm["Top1"], "-o", color='navy', label='FGSM Top 1')
axs[0].plot(fgsm["Epsilon"], fgsm["Top5"], "-s", color='navy', label='FGSM Top 5')
axs[0].plot(bim["Epsilon"], bim["Top1"], "-o", color='grey', label='BIM Top 1')
axs[0].plot(bim["Epsilon"], bim["Top5"], "-s", color='grey', label='BIM Top 5')
axs[0].legend()
axs[0].set_xlabel("Epsilon * 255", fontsize=20)

## Right: Confidences
axs[1].plot(fgsm["Epsilon"], fgsm["Confidence"], "-o", color='orange', label='FGSM Confidence')
axs[1].plot(bim["Epsilon"], bim["Confidence"], "-s", color='orange', label='BIM Confidence')

axs[1].legend()
axs[1].set_xlabel("Epsilon * 255", fontsize=20)

## Save plot
fig.tight_layout()
#fig.savefig("plots/Compare-attacks.pdf")
#fig.savefig("plots/Compare-attacks.png")

In [ ]:
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);

## References

[1] &emsp; Goodfellow et al. (2015) *Explaining and Harnessing Adversarial Examples*